# Lets take a look at this data

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [0]:
books_df = pd.read_csv("../data/raw/Books.csv", low_memory=False)
ratings_df = pd.read_csv("../data/raw/Ratings.csv", low_memory=False)
users_df = pd.read_csv("../data/raw/Users.csv", low_memory=False)

## Books

In [0]:
books_df.head()

In [0]:
# not needed for analysis
books_df.drop(['Image-URL-S', 'Image-URL-M', 'Image-URL-L'], axis=1, inplace=True)

In [0]:
books_df.columns = ['ISBN', 'bookTitle', 'bookAuthor', 'yearOfPublication', 'publisher']

Rename for ease of use later.

In [0]:
books_df.info()

`yearOfPublication` is the only one that looks a bit strange this should be an int or at least float. So lets check it out.

In [0]:
# check which entries in Year-Of-Publication are not numeric
books_df[books_df['yearOfPublication'].str.isnumeric() == False]


In [0]:
#ISBN '0789466953'
books_df.loc[books_df.ISBN == '0789466953','yearOfPublication'] = 2000
books_df.loc[books_df.ISBN == '0789466953','bookAuthor'] = "James Buckley"
books_df.loc[books_df.ISBN == '0789466953','publisher'] = "DK Publishing Inc"
books_df.loc[books_df.ISBN == '0789466953','bookTitle'] = "DK Readers: Creating the X-Men, How Comic Books Come to Life (Level 4: Proficient Readers)"

#ISBN '078946697X'
books_df.loc[books_df.ISBN == '078946697X','yearOfPublication'] = 2000
books_df.loc[books_df.ISBN == '078946697X','bookAuthor'] = "Michael Teitelbaum"
books_df.loc[books_df.ISBN == '078946697X','publisher'] = "DK Publishing Inc"
books_df.loc[books_df.ISBN == '078946697X','bookTitle'] = "DK Readers: Creating the X-Men, How It All Began (Level 4: Proficient Readers)"

#ISBN '2070426769'
books_df.loc[books_df.ISBN == '2070426769','yearOfPublication'] = 2003
books_df.loc[books_df.ISBN == '2070426769','bookAuthor'] = "Jean-Marie Gustave Le ClÃ?Â©zio"
books_df.loc[books_df.ISBN == '2070426769','publisher'] = "Gallimard"
books_df.loc[books_df.ISBN == '2070426769','bookTitle'] = "Peuple du ciel, suivi de 'Les Bergers"

In [0]:
books_df.loc[(books_df.ISBN == '0789466953') | (books_df.ISBN == '078946697X') | (books_df.ISBN == '2070426769'),:]

In [0]:
books_df.yearOfPublication = pd.to_numeric(books_df.yearOfPublication, errors='coerce')

### ISBN

In [0]:
books_df.ISBN.nunique() / books_df.shape[0]

### Book title

In [0]:
books_df.bookTitle = books_df.bookTitle.str.lower().str.strip()

In [0]:
books_df.bookTitle.nunique() / books_df.shape[0]

This alone isn't too concerning a book can have the same title as another. Lets also see about with the same author too.

In [0]:
books_df.bookAuthor = books_df.bookAuthor.str.lower().str.strip()

In [0]:
# check for entries with duplicate book titles and the same author
duplicated_books_same_author = books_df[
    books_df.duplicated(subset=['bookTitle', 'bookAuthor'], keep=False)
].sort_values(by=['bookTitle', 'bookAuthor'])

In [0]:
duplicated_books_same_author.head()

In [0]:
# drop duplicates just to count
dropped = books_df.drop_duplicates(subset=['bookTitle','bookAuthor'], keep='first', inplace=False)
print(dropped.bookTitle.nunique() / dropped.shape[0])

Improved by filtering based on books with the same author and title.

In [0]:
duplicated_books_same_author

So it seems most are for books with the same title and same author. Likely being published or republished again. This I'm not such a fan of and I will tidy up as I don't think the exact addition or publishing matters so much. Especially in a content-based recommender.

In [0]:
# sort by bookAge descending so that original is kept
books_df.sort_values(by='yearOfPublication', ascending=True, inplace=True)
books_df_unique = books_df.drop_duplicates(subset=['bookTitle','bookAuthor'], keep='first').copy()
isbn_mapping = {}

for key, group in books_df.groupby(['bookTitle','bookAuthor']):
    kept_isbn = group.ISBN.iloc[0]
    for _, row in group.iterrows():
        if row.ISBN != kept_isbn:
            isbn_mapping[row.ISBN] = kept_isbn

isbn_mapping_df = pd.DataFrame(list(isbn_mapping.items()), columns=['dropped_isbn', 'kept_isbn'])

In [0]:
# test isbn_mapping_df
dropped_isbn = isbn_mapping_df.iloc[0].dropped_isbn
kept_isbn = isbn_mapping_df.iloc[0].kept_isbn
books_df[books_df.ISBN.isin([dropped_isbn, kept_isbn])]

In [0]:
# TODO: temporary
books_df = books_df_unique

### Year of publication

In [0]:
print(sorted(books_df['yearOfPublication'].unique()))

The dataset was published in 2004 so some of these values greater than this look suspicious especially those greater than the current year!

In [0]:
print(books_df.yearOfPublication[books_df.yearOfPublication > 2006].value_counts().sort_index())

There aren't so many of these values so this is not so worrisome but nonetheless we'll set these to NAN and impute them.

In [0]:
books_df.loc[(books_df.yearOfPublication > 2006) | (books_df.yearOfPublication == 0),'yearOfPublication'] = np.NAN
books_df.yearOfPublication.fillna(round(books_df.yearOfPublication.mean()), inplace=True)
books_df.yearOfPublication = books_df.yearOfPublication.astype(np.int32)

For now I imputed the missing year of publication with the mean but I would likely use an API as I do later to get the true values as imputing is great but if you can get the real value from somewhere why not.

In [0]:
# I think age of book rather than year of publication is more relevant
CURRENT_YEAR = 2006
books_df['bookAge'] = CURRENT_YEAR - books_df.yearOfPublication
books_df.drop(['yearOfPublication'], axis=1, inplace=True)

In [0]:
# visualise the counts of bookAge
books_df.bookAge.hist(bins=100)
plt.title('Distribution of Book Age')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

In [0]:
# visualise the counts of bookAge
books_df.bookAge[books_df.bookAge <100].hist(bins=100)
plt.title('Distribution of Book Age')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

### Publishers

In [0]:
books_df.loc[books_df.publisher.isnull(),:]

There are two missing publishers. I could look into the data for this author and their usual publisher but instead I'll use this revolutionary and powerful tool called google - thx @ https://isbnsearch.org/.

In [0]:
#ISBN '193169656X'
books_df.loc[books_df.ISBN == '193169656X','publisher'] = 'NovelBooks, Inc.'

#ISBN '1931696993'
books_df.loc[books_df.ISBN == '1931696993', 'publisher'] = 'CreateSpace Independent Publishing Platform'

In [0]:
print(sorted(books_df['publisher'].unique()))

In [0]:
books_df.publisher.unique().size

In [0]:
books_df.publisher.str.lower().str.strip().unique().size

In [0]:
books_df.publisher = books_df.publisher.str.lower().str.strip()

In [0]:
(books_df.publisher.unique().size / books_df.publisher.size) * 100

### Authors

In [0]:
books_df.loc[books_df.bookAuthor.isnull(),:]


Even google could not find the author to this book so we'll set it to unkown.

In [0]:
books_df.loc[books_df.ISBN == '9627982032','bookAuthor'] = "Unknown"

In [0]:
books_df['bookAuthor'] = books_df.bookAuthor.str.lower().str.strip()

In [0]:
books_df.isnull().sum()

In [0]:
books_df.to_csv('../data/processed/books.csv', index=False)

For now we'll move onto the next dataset

## Users

I decided fairly early on I would not be looking to use users in my modelling at this stage since it provides very little information and is unlikely to influence my predictions too much in the global world we live in. Although age might be a useful predictor this is likely baked into the user history of books and I also presume such a tool/model as I'm making will likely be used by adults in a certain age range.

In [0]:
users_df.head()

In [0]:
users_df.columns = ['userID', 'location', 'age']

In [0]:
users_df.info()

In [0]:
users_df.userID.unique().size / users_df.userID.size

### Age

In [0]:

plt.figure(figsize=(10, 7))
users_df.age.hist(bins=40, color='salmon')

In [0]:
# assume only people between 5 and 95 read books or are at least interested in using a book recommender system
users_df.loc[(users_df.age > 95) | (users_df.age < 5), 'Age'] = np.nan
users_df.age = users_df.age.fillna(users_df.age.mean())
users_df.age = users_df.age.astype(np.int32)

### Location

In [0]:
users_df.location

It looks like the location for each user contains city, region/state, country. 
For simplicity at this first stage I will just extract the country.

In [0]:
# extract the country from the last comma separated value of the location field
users_df['country'] = users_df.location.apply(lambda x: x.split(',')[-1].strip())

In [0]:
users_df['country'].unique().size

Seeming as there aren't 709 countries in the world this isn't the best. Hopefully using a library to standardise these will help.

In [0]:
# import country_converter as coco
# converted_country=coco.convert(names=users_df["country"], to="ISO3")
# users_df['country'] = converted_country

In [0]:
# users_df['country'].isnull().sum()

In [0]:
# import country_converter as coco
# converted_country=coco.convert(names=users_df["country"][0:1000], to="ISO3")

## Ratings

In [0]:
ratings_df.head()

In [0]:
ratings_df.columns = ['userID', 'ISBN', 'bookRating']

In [0]:
print(f"Number of ratings: {ratings_df.shape[0]}")
print(f"Number of unique users: {ratings_df.userID.unique().size}")
print(f"Number of unique books: {ratings_df.ISBN.unique().size}")
print(f"Number of ratings if each user rated each book: {users_df.shape[0] * books_df.shape[0]}")
print(f"Percentage of ratings: {ratings_df.shape[0] / (users_df.shape[0] * books_df.shape[0]) * 100}%")

We have a very sparse matrix most users naturally haven't read or rated most books. This is a common challenge with recommender systems.

In [0]:
print(f"Books without ratings {books_df[~books_df.ISBN.isin(ratings_df.ISBN)].shape[0]}")
print(f"Ratings for books that we don't have {ratings_df[~ratings_df.ISBN.isin(books_df.ISBN)].shape[0]}")

### Dealing with duplicate books

As I removed "duplicate" books earlier I decided to remap the ISBNs of the removed books to the ISBN of the book I kept as I am deeming the books equivalent and consequently the rating too.

In [0]:
# remap all removed ISBNs to the kept ISBN
ratings_df = ratings_df.merge(isbn_mapping_df, left_on='ISBN', right_on='dropped_isbn', how='left')
ratings_df.head(15)

In [0]:
# also merge the kept_isbn column only on ISBN
ratings_df = ratings_df.merge(isbn_mapping_df[['kept_isbn']], left_on='ISBN', right_on='kept_isbn', how='left')

In [0]:
ratings_df.isnull().sum() / ratings_df.shape[0]

In [0]:
print(f"Books without ratings {books_df[~books_df.ISBN.isin(ratings_df.ISBN)].shape[0]}")
print(f"Ratings for books that we don't have {ratings_df[~ratings_df.ISBN.isin(books_df.ISBN)].shape[0]}")

In [0]:
# replace the ISBN with the kept_isbn_x when dropped_isbn and kept_isbn_x are both not null
ratings_df['ISBN'] = np.where((ratings_df['dropped_isbn'].notnull()) & (ratings_df['kept_isbn_x'].notnull()), ratings_df['kept_isbn_x'], ratings_df['ISBN'])
ratings_df.head(15)


In [0]:
print(f"Books without ratings {books_df[~books_df.ISBN.isin(ratings_df.ISBN)].shape[0]}")
print(f"Ratings for books that we don't have {ratings_df~ratings_df.ISBN.isin(books_df.ISBN)].shape[0]}")

There are now much fewer ratings for books that we don't have by doing this remapping.

### Dropping ratings and books

I drop all ratings that are for books not in the books dataset and all books that don't have any ratings in the ratings dataset.

In [0]:
# drop rows for ratings which we don't have the book
ratings_df = ratings_df[ratings_df['ISBN'].isin(books_df['ISBN'])]
ratings_df.head(15)

In [0]:
ratings_df.shape[0]

In [0]:
ratings_df = ratings_df.drop(['dropped_isbn', 'kept_isbn_x', 'kept_isbn_y'], axis=1)

In [0]:
# we need to remove ratings for books not in the books dataset and from users not in the users dataset
print(f"Ratings from users that don't exist {ratings_df[~ratings_df.userID.isin(users_df.userID)].shape[0]}")
print(f"Ratings for books that we don't have {ratings_df[~ratings_df.ISBN.isin(books_df.ISBN)].shape[0]}")
print(f"Books without ratings {books_df[~books_df.ISBN.isin(ratings_df.ISBN)].shape[0]}")

In [0]:
# now drop books without ratings
books_df = books_df[books_df.ISBN.isin(ratings_df.ISBN)]

In [0]:
rating_counts = ratings_df.bookRating.value_counts().sort_index()
colors = ['lightgrey' if rating == 0 else 'salmon' for rating in rating_counts.index]

plt.figure(figsize=(10, 7))
rating_counts.plot.bar(color=colors)
plt.xlabel('Book Rating')
plt.ylabel('Frequency')
plt.title('Book Ratings: Implicit (0) and Explicit (1–10)', color='black')
plt.show()

Most ratings are implicit with value 0.

In [0]:
plt.figure(figsize=(10, 7))
ratings_df[ratings_df.bookRating > 0].bookRating.value_counts().sort_index().plot.bar(color='salmon')
plt.xlabel('Book Rating')
plt.ylabel('Frequency')

In [0]:
print(f"Books without ratings {books_df[~books_df.ISBN.isin(ratings_df.ISBN)].shape[0]}")

In [0]:
ratings_df.to_csv('../data/processed/ratings_d.csv', index=False)
books_df.to_csv('../data/processed/books.csv', index=False)